In [1]:
import torch
from algo_reasoning.src.sampler import CLRSDataset
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/ronald/Documents/env/algo_reasoning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def homophily_measure(adj_matrix, classes):
    """
    Compute the homophily measure of the graph
    """
    n = adj_matrix.shape[0]
    homophily = 0
    
    for i in range(n):
        node_homophily = 0
        for j in range(n):
            if classes[i] == classes[j]:
                node_homophily += adj_matrix[i, j]

        node_homophily /= adj_matrix[i].sum()
        homophily += node_homophily

    homophily /= n
    return homophily

In [3]:
algorithms = ["articulation_points"]

nb_nodes = 16
batch_size = 8
ds = CLRSDataset(algorithms, nb_nodes, batch_size, 1000, seed=7)

In [5]:
obj = next(iter(ds))

In [26]:
classes = obj.outputs.is_cut[3]
adj_matrix = obj.inputs.A[3]

In [27]:
homophily_measure(adj_matrix, classes)

tensor(0.8729)